# Week 41

### OLS gradient descent

In [ ]:
import numpy as np

In [ ]:
class NumpyLinRegClass():

    def __init__(self, bias=-1, dim_hidden=0):
        self.bias=bias
        
    
    def fit(self, X_train, t_train, X_val=None, t_val=None, eta = 0.1, epochs=10, tol=0.001):

        if self.bias:
            X = add_bias(X_train, self.bias)

        (N, m) = X.shape
        
        self.weights = weights = np.zeros(m)
        
        if (X_val is None) or (t_val is None): 
            for e in range(epochs):
                weights -= eta / N *  X.T @ (X @ weights - t_train)  
    
        else:
            self.loss = np.zeros(epochs)
            self.accuracies = np.zeros(epochs)
            for e in range(epochs):
                weights -= eta / N *  X.T @ (X @ weights - t_train)   
                self.loss[e] =          MSE(X @ self.weights, t_train)
                self.accuracies[e] =    accuracy(self.predict(X_val), t_val)



    def predict(self, X, threshold=0.5):
        if self.bias:
            X = add_bias(X, self.bias)

        ys = X @ self.weights
        return ys > threshold

In [ ]:
class NumpyLogReg():


    def __init__(self, bias=-1, dim_hidden=0):
        self.bias=bias

        
    def fit(self, X_train, t_train, X_val=None, t_val=None, eta = 0.1, epochs=10, tol=0.01, n_epochs_no_update=5):
        
        (N, m) = X_train.shape
        if self.bias:
            X = add_bias(X_train, self.bias)
        

        self.weights = weights = np.zeros(m+1)
        

        if (X_val is None) or (t_val is None):       
            for e in range(epochs):
                weights -= eta / N *  X.T @ (self.forward(X) - t_train)  
        
        else:
            self.loss = np.zeros(epochs)
            self.val_loss = np.zeros(epochs)
            self.accuracies = np.zeros(epochs)
            self.val_accuracies = np.zeros(epochs)
            self.epochs_ran = epochs
            # loop trough first n_epochs_no_update
            for e in range(n_epochs_no_update):
                weights -= eta / N *  X.T @ (self.forward(X) - t_train)      
                self.loss[e] =          CE(self.predict_probability(X_train), t_train)
                self.val_loss[e] =      CE(self.predict_probability(X_val), t_val)
                self.val_accuracies[e]= accuracy(self.predict(X_val), t_val)
                self.accuracies[e] =    accuracy(self.predict(X_train), t_train)
            # loop trough rest
            for e in range(n_epochs_no_update, epochs):
                weights -= eta / N *  X.T @ (self.forward(X) - t_train)      
                self.loss[e] =          CE(self.predict_probability(X_train), t_train)
                self.val_loss[e] =      CE(self.predict_probability(X_val), t_val)
                self.val_accuracies[e] =accuracy(self.predict(X_val), t_val)
                self.accuracies[e] =    accuracy(self.predict(X_train), t_train)
                # if no update exit training
                if self.loss[e-n_epochs_no_update] - self.loss[e] < tol:
                    self.epochs_ran = e
                    self.loss = self.loss[:e]
                    self.val_loss = self.val_loss[:e]
                    self.val_accuracies = self.val_accuracies[:e]
                    self.accuracies = self.accuracies[:e]
                    break

            
    
    def forward(self, X):
        return logistic(X @ self.weights)
    

    def predict(self, x, threshold=0.5):
        if self.bias:
            x = add_bias(x,self.bias)
        return (self.forward(x) > threshold).astype('int')
    
    def predict_probability(self, x):
        if self.bias:
            x = add_bias(x,self.bias)
        return (self.forward(x))

In [ ]:
class NumpyLogRegMulti():


    def __init__(self, bias=-1, dim_hidden=0):
        self.bias=bias

        
    def fit(self, X_train, t_train, X_val=None, t_val=None, eta = 0.1, epochs=10, tol=0.01, n_epochs_no_update=5):
        
        (N, m) = X_train.shape
        t = one_hot_encoding(t_train)

        if self.bias:
            X = add_bias(X_train, self.bias)
        

        self.weights = weights = np.zeros((m+1, t.shape[1]))
        

        if (X_val is None) or (t_val is None):      
            for e in range(epochs):
                weights -= eta / N *  X.T @ (self.forward(X) - t)  


        else: # if validation is provided: calculate loss and accuracies for each epoch
            t_val_t = one_hot_encoding(t_val)
            self.loss = np.zeros(epochs)
            self.val_loss = np.zeros(epochs)
            self.accuracies = np.zeros(epochs)
            self.val_accuracies = np.zeros(epochs)
            self.epochs_ran = epochs
            # loop trough first n_epochs_no_update
            for e in range(n_epochs_no_update):
                weights -= eta / N *  X.T @ (self.forward(X) - t)      
                self.loss[e] =          sum(CE(self.predict_probability(X_train), t))
                self.val_loss[e] =      sum(CE(self.predict_probability(X_val), t_val_t))
                self.val_accuracies[e]= accuracy(self.predict(X_val), t_val)
                self.accuracies[e] =    accuracy(self.predict(X_train), t_train)
            # loop trough rest
            for e in range(n_epochs_no_update, epochs):
                weights -= eta / N *  X.T @ (self.forward(X) - t)      
                self.loss[e] =          sum(CE(self.predict_probability(X_train), t))
                self.val_loss[e] =      sum(CE(self.predict_probability(X_val), t_val_t))
                self.val_accuracies[e] =accuracy(self.predict(X_val), t_val)
                self.accuracies[e] =    accuracy(self.predict(X_train), t_train)
                # if no update exit training
                if self.loss[e-n_epochs_no_update] - self.loss[e] < tol:
                    self.epochs_ran = e
                    self.loss = self.loss[:e]
                    self.val_loss = self.val_loss[:e]
                    self.val_accuracies = self.val_accuracies[:e]
                    self.accuracies = self.accuracies[:e]
                    break

            
    
    
    def forward(self, X):
        return logistic(X @ self.weights)
    
    def predict(self, x, threshold=0.5):
        if self.bias:
            x = add_bias(x,self.bias)
        return self.forward(x).argmax(axis=1)
    
    def predict_probability(self, x):
        if self.bias:
            x = add_bias(x,self.bias)
        return (self.forward(x))
    

    def score(self,x,t):
        return accuracy(self.predict(x), t)

In [22]:
# Importing various packages
import numpy as np
import pandas as pd

np.random.seed(2020)

learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.0, 0.1, 0.5, 0.9]
change = 0.0
mse = []

# the number of datapoints
for i in range(len(momentum)):
    mse_tmp = []
    for j in range(len(learning_rate)):
        polydegree = 2
        n = 100
        x = 2*np.random.rand(n,1)
        y = 4+3*x**2+np.random.randn(n,1)

        X = np.c_[np.ones((n,1)), x, x**2] 

        beta = np.random.randn(polydegree+1,1)

        eta = learning_rate[j]#learning rate
        Niterations = 1000

        for iter in range(Niterations):
            gradient = (2.0/n)*X.T @ (X @ beta-y)
            change = eta*gradient + momentum[i]*change
            beta -= change

        
        ypredict = X.dot(beta)

        mse_tmp.append(np.round((1.0/n)*np.sum((y - X@beta)**2), 3))
        
    mse.append(mse_tmp)

beta_linreg = np.linalg.inv(X.T @ X) @ X.T @ y
mse_predict = (1.0/n)*np.sum((y - X@beta_linreg)**2)
print("MSE (linreg): ", mse_predict)

panda = pd.DataFrame(data=mse)
print(panda)
print("\nmin mse:",np.min(mse))

MSE (linreg):  0.8853271991907216
        0      1      2      3
0  23.624  1.391  1.068  0.726
1  11.550  1.405  0.835  0.671
2   6.413  1.002  1.135  0.942
3   1.480  1.071  0.880  0.885

min mse: 0.671


We see that a low learning rate results in bad MSEs probably due to a relatively low number of iterations and it not converging. With higher momentum it drastically improves while the higher learning rates at best get minor improvements.


### Ridge gradient descent

In [23]:
# Importing various packages
import numpy as np
import pandas as pd


learning_rate = [ 0.001, 0.01, 0.1, 0.25]
momentum = [0.0, 0.1, 0.5, 0.9]
change = 0.0
mse = []
lamda_list = [0.0001, 0.001, 0.01, 0.1, 1.0]


# the number of datapoints
for i in range(len(momentum)):
    mse_tmp = []
    for j in range(len(learning_rate)):
        mse_tmp_tmp = []
        for k in range(len(lamda_list)):
            polydegree = 2
            n = 100
            x = 2*np.random.rand(n,1)
            y = 4+3*x**2+np.random.randn(n,1)

            X = np.c_[np.ones((n,1)), x, x**2] 

            beta = np.random.randn(polydegree+1,1)

            eta = learning_rate[j]#learning rate
            Niterations = 1000

            lamda = lamda_list[k]

            for iter in range(Niterations):
                gradient = (2.0/n)*X.T@(X@beta - y) + 2*lamda*beta
                change = eta*gradient + momentum[j]*change
                beta -= change

            ypredict = X.dot(beta)

            mse_tmp_tmp.append(np.round((1.0/n)*np.sum((y - X@beta)**2), 3))
        
        mse_tmp.append(mse_tmp_tmp)
        
    mse.append(mse_tmp)

beta_linreg = np.linalg.inv(X.T @ X) @ X.T @ y
mse_predict = (1.0/n)*np.sum((y - X@beta_linreg)**2)
print("MSE (linreg): ", mse_predict)

for i in range(len(mse)):
    panda = pd.DataFrame(data=mse[i])
    print("momentum = ", momentum[i])  
    print(panda)

print("\nmin mse:",np.min(mse))

MSE (linreg):  0.9188629578817374
momentum =  0.0
       0      1      2      3      4
0  1.280  1.134  1.758  1.899  3.807
1  1.257  0.894  0.817  1.293  3.992
2  1.011  0.987  1.113  1.066  3.738
3  0.948  0.912  0.994  0.922  4.039
momentum =  0.1
       0      1      2      3      4
0  1.180  1.219  1.076  0.866  4.072
1  1.165  1.219  0.852  1.097  3.697
2  1.133  1.146  0.861  0.864  4.150
3  0.993  0.974  0.949  1.322  3.527
momentum =  0.5
       0      1      2      3      4
0  1.275  1.239  1.087  1.422  3.910
1  1.126  1.116  1.099  1.069  4.459
2  1.064  0.983  0.900  1.134  3.664
3  0.840  0.959  0.963  0.943  3.999
momentum =  0.9
       0      1      2      3      4
0  1.048  1.687  1.173  1.715  4.238
1  1.000  1.003  0.745  1.239  3.723
2  0.756  1.128  0.933  1.083  3.775
3  1.001  0.777  0.694  0.950  3.903

min mse: 0.694


Skrive noe her om det over:::::

### OLS stochastic gradient descent

In [21]:
# Importing various packages
import numpy as np
import pandas as pd

mse = []
epochs = [1, 10, 50, 100, 500, 1000]

polydegree = 2
n = 100
x = 2*np.random.rand(n,1)
y = 4+3*x**2+np.random.randn(n,1)
X = np.c_[np.ones((n,1)), x, x**2] 

H = (2.0/n)* X.T @ X
EigValues, EigVectors = np.linalg.eig(H)

for n_epochs in epochs:

    beta = np.random.randn(polydegree+1,1)
    eta = 0.01/np.max(EigValues)
    M = 5   #size of each minibatch
    m = int(n/M) #number of minibatches
    t0, t1 = 5, 50

    def learning_schedule(t):
        return t0/(t+t1)

    for epoch in range(n_epochs):
    # selects a random mini-batch at every epoch. it does not garanty that all the data will be used
        for i in range(m):
            random_index = M*np.random.randint(m)
            xi = X[random_index:random_index+M]
            yi = y[random_index:random_index+M]
            gradients = (2.0/M)* xi.T @ ((xi @ beta)-yi)
            eta = learning_schedule(epoch*m+i)
            beta = beta - eta*gradients
    
    mse.append((1.0/n)*np.sum((y - X@beta)**2))

beta_linreg = np.linalg.inv(X.T @ X) @ X.T @ y
mse_predict = (1.0/n)*np.sum((y - X@beta_linreg)**2)
print("MSE (linreg): ", mse_predict)

panda = pd.DataFrame(data=mse)
print(panda)

print("\nmin mse:",np.min(mse))

MSE (linreg):  0.8742291237044855
          0
0  1.382672
1  0.893757
2  0.931771
3  0.907148
4  0.909486
5  0.920278

min mse: 0.8937572347487786


In [ ]:
Skrive noe her om det over:::::

### Ridge stochastic gradient descent

In [20]:
# Importing various packages
import numpy as np
import pandas as pd

mse = []
epochs = [1, 10, 50, 100, 500]

mse = []
lamda_list = [0.0001, 0.001, 0.01, 0.1, 1.0]

polydegree = 2
n = 100
x = 2*np.random.rand(n,1)
y = 4+3*x**2+np.random.randn(n,1)
X = np.c_[np.ones((n,1)), x, x**2] 

H = (2.0/n)* X.T @ X
EigValues, EigVectors = np.linalg.eig(H)

for lamda in lamda_list:
    mse_tmp = []
    for n_epochs in epochs:
        beta = np.random.randn(polydegree+1,1)
        eta = 0.01/np.max(EigValues)
        M = 5   #size of each minibatch
        m = int(n/M) #number of minibatches
        t0, t1 = 5, 50

        def learning_schedule(t):
            return t0/(t+t1)

        for epoch in range(n_epochs):
        # selects a random mini-batch at every epoch. it does not garanty that all the data will be used
            for i in range(m):
                random_index = M*np.random.randint(m)
                xi = X[random_index:random_index+M]
                yi = y[random_index:random_index+M]
                gradients = (2.0/n)*X.T@(X@beta - y) + 2*lamda*beta
                eta = learning_schedule(epoch*m+i)
                beta = beta - eta*gradients
        
        mse_tmp.append((1.0/n)*np.sum((y - X@beta)**2))
    mse.append(mse_tmp)

beta_linreg = np.linalg.inv(X.T @ X) @ X.T @ y
mse_predict = (1.0/n)*np.sum((y - X@beta_linreg)**2)
print("MSE (linreg): ", mse_predict)

print("mse.hape", np.shape(mse))

panda = pd.DataFrame(data=mse)
print(panda)

print("\nmin mse:",np.min(mse))

MSE (linreg):  0.9901289773595197
mse.hape (5, 5)
          0         1         2         3         4
0  1.093200  1.108377  1.037154  1.031901  0.994656
1  1.174978  1.095579  1.144712  1.003394  1.011738
2  1.283342  0.995774  1.029529  1.014229  1.021625
3  1.150841  1.167716  1.166153  1.166359  1.166051
4  3.762844  3.758817  3.758817  3.758817  3.758817

min mse: 0.9946562652697165
